In [ ]:
!pip install ekphrasis
!pip install transformers==4.2.1
!pip install tf-models-official
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter
from official import nlp
import official.nlp.optimization

import ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 184kB 7.9MB/s 
     |████████████████████████████████| 71kB 5.1MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=7d81c7ce5cc5dc1dd29e7c9f5ba8699e97d003d7dfbdbfb8d7b0c47e5a1ec3f6
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=9e58800aa7298531f9b7923a2fa61ec1365e5f7edd5a403e58b42e1ff2eaa8a3
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
Successfully built ekphrasis ftfy
     |████████████████████████████████| 1.8MB 5.9MB/s 
     |████████████████████████████████| 2.9MB 19.2MB/s 
     |████████████████████████████████| 870kB 40.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=10ebf5c4ba3be7e633baf7

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.1.29.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.1.29.146:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [ ]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... 

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/data'
!ls


/content/drive/MyDrive/data
albert-1-b.csv	   albert-2.csv       answer-1-b.csv	    task2.csv
albert-1-b.gsheet  albert-2.gsheet    dev		    test
albert-1-c.csv	   answer-1-a.csv     Dexter_answer.gsheet  train
albert-1-c.gsheet  answer-1-a.gsheet  slang.txt


In [ ]:
df_train = pd.read_csv('./train/task_1_b_c.csv', encoding='utf-8')
df_train['humor_rating'] = df_train['humor_rating'].fillna(0)
df_train['humor_controversy'] = df_train['humor_controversy'].fillna(2)
text_train = df_train["text"]
df_train.head(15)

df_val = pd.read_csv('./dev/task_1_b_c_dev.csv', encoding='utf-8')
df_val['humor_rating'] = df_val['humor_rating'].fillna(0)
df_val['humor_controversy'] = df_val['humor_controversy'].fillna(2)
text_val = df_val["text"]
df_val.head(15)

df_test = pd.read_csv('./test/public_test.csv', encoding='utf-8')
text_test = df_test["text"]
df_test.head(15)

,id,text
0,9001,Finding out your ex got fat is like finding 20...
1,9002,"For Brockmann, stereotypes imperil national se..."
2,9003,A girl runs up to her mother with a pile of cr...
3,9004,gotta wonder if baseball still would've been c...
4,9005,When you're dreading getting in the shower cuz...
5,9006,Parenthood feels like you're on a magical unic...
6,9007,It's Friday night and I'm out of control! Gett...
7,9008,me: I didn't know this was a hibachi restauran...
8,9009,Maybe we need to turn the United States off an...
9,9010,I asked my North Korean friend how it was ther...


In [ ]:
print(len(text_train))
print(len(text_val))
print(len(text_test))

4932
632
1000


In [ ]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()
print_text(text_train,0,10)
print("##############################################################################################################")
print_text(text_val,0,10)

TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee walks into the room* TENNESSEE: Oh shit...

A man inserted an advertisement in the classifieds "Wife Wanted". The next day, he received 1000 of replies, all reading: "You can have mine." Free delivery also available at your door step

How many men does it take to open a can of beer? None. It should be open by the time she brings it to the couch.

Told my mom I hit 1200 Twitter followers. She pointed out how my brother owns a house and I'm wanted by several collection agencies. Oh ma!

Roses are dead. Love is fake. Weddings are basically funerals with cake.

ME: I'm such an original. Truly one of a kind. ALSO ME: [holding a glass of white wine] I love to laugh.

What's the difference between a Mormon man and a Muslim man. A Mormon man gets 72 virgins and THEN kills himself.

Stop calling 9-1-1 because you've run out of toilet paper. Yes. If you're running out of toilet paper don't dial 9-1-1. The number for that is 

In [ ]:
# Functions for chat word conversion
f = open("./slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("-")[0]
        cw_expanded = line.split("-")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)
print(chat_words_list)
def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

{'vip  ', 'itt  ', 'wats  ', 'wwut  ', 'fmnb  ', 'ghei  ', 'bz  ', 'fiic  ', '4g8  ', 'h4x0r  ', 'newb  ', 'bowt  ', 'lu2  ', 'wen  ', 'wrk  ', 'csi  ', 'ikwum  ', 'isd  ', 'pwnzor  ', 'roflcopters  ', 'diez  ', 'whteve  ', 'tcfc  ', 'swakaah  ', 'jfj  ', 'irly  ', 'ltnc  ', 's2bu  ', 'wogge  ', 'ddf  ', 'd2m  ', 'wtfs  ', 'tfbundy  ', 'outa  ', 'bfitww  ', 'phuck  ', 'ntk  ', 'soafb  ', 'gpb  ', 'dyac  ', 'wuwtab  ', 'nvm  ', 'ive  ', 'til  ', 'ilykthnxbai  ', 'waygow  ', 'seks  ', 'ttmn  ', 'heh  ', 'lml  ', 'smhb  ', 'rdy  ', 'roflmgdmfao  ', 'dycotfc  ', 'ybya  ', 'code 8  ', 'nethin  ', 'oom  ', 'nic  ', 'but6  ', 'hlo  ', 'odtaa  ', 'wrud  ', '2moz  ', 'bbiam  ', 'urht  ', 'brt  ', 'o  ', 'beotch  ', 'cp  ', 'gfad  ', 'bfn  ', 'gfx  ', 'retrotextual  ', 'rotflmfao  ', 'fxe  ', 'urafb  ', 'yur  ', 'lv  ', '31337  ', 'compy  ', 'gokw  ', 'hfn  ', 'mofo  ', 'gtgmmiloms  ', 'itn  ', 'nop  ', 'lobl  ', 'sotc  ', 'nmhu  ', 'hpoa  ', 'seg  ', 'myob  ', 'b2b  ', 'q33r  ', 'wgaf  ', 'gosa

In [ ]:
# Chat word conversion
# Training set
text_train = text_train.apply(lambda text: chat_words_conversion(text))
print_text(text_train,0,10)

print("********************************************************************************")

# Validation set
text_val = text_val.apply(lambda text: chat_words_conversion(text))
print_text(text_val,0,10)

# Test set
text_test = text_test.apply(lambda text: chat_words_conversion(text))
# print_text(text_test,0,10)

TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee walks into the room* TENNESSEE: Oh shit...

A man inserted an advertisement in the classifieds "Wife Wanted". The next day, he received 1000 of replies, all reading: "You can have mine." Free delivery also available at your door step

How many men does it take to open a can of beer? None. It should be open by the time she brings it to the couch.

Told my mom I hit 1200 Twitter followers. She pointed out how my brother owns a house and I'm wanted by several collection agencies. Oh ma!

Roses are dead. Love is fake. Weddings are basically funerals with cake.

ME: I'm such an original. Truly one of a kind. ALSO ME: [holding a glass of white wine] I love to laugh.

What's the difference between a Mormon man and a Muslim man. A Mormon man gets 72 virgins and THEN kills himself.

Stop calling 9-1-1 because you've run out of toilet paper. Yes. If you're running out of toilet paper don't dial 9-1-1. The number for that is 

In [ ]:

def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence

In [ ]:

# Training set
text_train = text_train.apply(lambda text: ekphrasis_pipe(text))
print("Training set completed.......")
#Validation set
text_val = text_val.apply(lambda text: ekphrasis_pipe(text))
print("Validation set completed.......")
#Test set
text_test = text_test.apply(lambda text: ekphrasis_pipe(text))
print("Test set completed.......")

Training set completed.......
Validation set completed.......
Test set completed.......


In [ ]:
# Finding length of longest array
maxLen = len(max(text_train,key = lambda text: len(text.split(" "))).split(" "))
print(maxLen)

132


In [ ]:

u = lambda text: len(text.split(" "))
sentence_lengths = []
for x in text_train:
    sentence_lengths.append(u(x))
print(sorted(sentence_lengths)[-50:])
print(len(sentence_lengths))

[65, 65, 65, 66, 66, 66, 66, 66, 66, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 68, 68, 68, 69, 69, 69, 69, 69, 69, 70, 71, 71, 71, 71, 72, 72, 73, 73, 73, 73, 74, 74, 75, 75, 76, 76, 76, 79, 83, 90, 132]
4932


In [ ]:
is_humor = df_train["is_humor"]
humor_rating = df_train["humor_rating"]
humor_controversy = df_train["humor_controversy"].astype(int)
offense_rating = df_train["offense_rating"]
print(Counter(is_humor))
print(Counter(humor_controversy))

Counter({1: 4932})
Counter({0: 2467, 1: 2465})


In [ ]:
is_humor_val = df_val["is_humor"]
humor_rating_val = df_val["humor_rating"]
humor_controversy_val = df_val["humor_controversy"].astype(int)
offense_rating_val = df_val["offense_rating"]
print(Counter(is_humor_val))
print(Counter(humor_controversy_val))

Counter({1: 632})
Counter({0: 324, 1: 308})


In [ ]:
from transformers import RobertaTokenizerFast, TFRobertaModel, TFBertModel, BertTokenizerFast, ElectraTokenizerFast, TFElectraModel, AlbertTokenizerFast, TFAlbertModel, XLNetTokenizerFast, TFXLNetModel, MPNetTokenizerFast, TFMPNetModel
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
text_train = list(text_train)
text_val = list(text_val)
text_test = list(text_test)

In [ ]:
train_encodings = tokenizer(text_train, max_length=150, truncation=True, padding="max_length", return_tensors='tf')
val_encodings = tokenizer(text_val, max_length=150, truncation=True, padding="max_length", return_tensors='tf')
test_encodings = tokenizer(text_test, max_length=150, truncation=True, padding="max_length", return_tensors='tf')

In [ ]:
print(np.shape(train_encodings["input_ids"]))
print(np.shape(val_encodings["input_ids"]))
print(np.shape(test_encodings["input_ids"]))

(4932, 150)
(632, 150)
(1000, 150)


In [ ]:

print(train_encodings["input_ids"][0])
print("***************************************************************************")
print(val_encodings["input_ids"][0])

tf.Tensor(
[  101  1026  2035 17695  2015  1028  5298  1026  1013  2035 17695  2015
  1028  1024  2057  2024  1996  2190  2110  1012  6343  2130  3310  2485
  1012  1008  5408  2791  4402  7365  2046  1996  2282  1008  1026  2035
 17695  2015  1028  5298  1026  1013  2035 17695  2015  1028  1024  2821
  4485  1012  1026  5567  1028   102     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int32)
***********************************************

In [ ]:
def hahackathon_task_1(input_shape):

    model = TFBertModel.from_pretrained('bert-base-uncased')
    layer = model.layers[0]
    #Model
    inputs = keras.Input(shape=input_shape, dtype='int32')
    input_masks = keras.Input(shape=input_shape, dtype='int32')

    outputs = layer([inputs, input_masks])
    output = outputs[0]
    # pooled_output = outputs[1]
    pooled_output = output[:, 0, :] # Use for bert, roberta, albert, mpnet, electra
    # pooled_output = output[:, -1] # Use for XLNet

    # Humour controversy
    humor_binary = layers.Dropout(0.3)(pooled_output)
    humor_binary = layers.Dense(1, activation="sigmoid")(humor_binary)

    model = keras.Model(inputs=[inputs,input_masks], outputs=humor_binary, name='Haha')
    
    return model

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():
    np.random.seed(45)
    model = hahackathon_task_1((150,))
    optimizer = keras.optimizers.Adam(learning_rate=2e-5)
    loss_fun = [
          tf.keras.losses.BinaryCrossentropy()
    ]
    metric = [
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=metric)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
model.summary()

Model: "Haha"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 768)          0           bert[0][0]                    

In [ ]:
checkpoint = ModelCheckpoint(filepath='/content/task-1-c-model.{epoch:03d}.h5',
                                 verbose = 0,
                                 save_weights_only=True,
                                 epoch=4)

In [ ]:
import time
start_time = time.time()
history_task_1_c = model.fit(
    x = [train_encodings["input_ids"], train_encodings["attention_mask"]],
    y = humor_controversy,
    validation_data = ([val_encodings["input_ids"],val_encodings["attention_mask"]], humor_controversy_val),
    callbacks = [checkpoint],
    batch_size=16,
    shuffle=True,
    epochs=4)
print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/4


309/309 [==============================] - 100s 104ms/step - loss: 0.7229 - binary_accuracy: 0.5116 - precision: 0.5024 - recall: 0.4764 - val_loss: 0.7082 - val_binary_accuracy: 0.4889 - val_precision: 0.4881 - val_recall: 0.9968
Epoch 2/4
309/309 [==============================] - 26s 85ms/step - loss: 0.6830 - binary_accuracy: 0.5576 - precision: 0.5543 - recall: 0.5523 - val_loss: 0.7019 - val_binary_accuracy: 0.5269 - val_precision: 0.6098 - val_recall: 0.0812
Epoch 3/4
309/309 [==============================] - 26s 84ms/step - loss: 0.6099 - binary_accuracy: 0.6632 - precision: 0.6627 - recall: 0.6345 - val_loss: 0.7882 - val_binary_accuracy: 0.5222 - val_precision: 0.5063 - val_recall: 0.7825
Epoch 4/4
309/309 [==============================] - 26s 83ms/step - loss: 0.3077 - binary_accuracy: 0.8688 - precision: 0.8650 - recall: 0.8748 - val_loss: 1.0639 - val_binary_accuracy: 0.5522 - val_precision: 0.5436 - val_recall: 0.5065
--- 188.71810936927795 seconds ---


In [1]:

val_answer = model.predict([val_encodings["input_ids"],val_encodings["attention_mask"]])

NameError: ignored

In [ ]:
val_humor_controversy = []

for i in range(0,len(val_answer)):
    if val_answer[i] > 0.5:
        val_humor_controversy.append(1)
    else:
        val_humor_controversy.append(0)

In [ ]:
con_mat_humor_controversy = tf.math.confusion_matrix(humor_controversy_val, val_humor_controversy, num_classes=None, weights=None, dtype=tf.dtypes.int32)
print(con_mat_humor_controversy)

tf.Tensor(
[[192 132]
 [152 156]], shape=(2, 2), dtype=int32)


In [ ]:
print(classification_report(humor_controversy_val, val_humor_controversy, digits=3))

              precision    recall  f1-score   support

           0      0.558     0.593     0.575       324
           1      0.542     0.506     0.523       308

    accuracy                          0.551       632
   macro avg      0.550     0.550     0.549       632
weighted avg      0.550     0.551     0.550       632



In [ ]:
import time
start_time = time.time()
test_answer = model.predict([test_encodings["input_ids"],test_encodings["attention_mask"]])
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.285627841949463 seconds ---


In [ ]:
test_1_c = []

for i in range(0,len(test_answer)):
    if test_answer[i] > 0.5:
        test_1_c.append(1)
    else:
        test_1_c.append(0)

In [ ]:
len(test_1_c)

1000

In [ ]:

test_id = df_test["id"]

In [ ]:
test_dict = {
    "id" : test_id,
    "humor_controversy" : test_1_c
}

In [ ]:
df_test = pd.DataFrame(test_dict)
df_test.head()

,id,humor_controversy
0,9001,0
1,9002,0
2,9003,1
3,9004,0
4,9005,0


In [ ]:
df_test.to_csv('albert-1-c.csv', index=False)